In [1]:
from gensim.models import KeyedVectors
w2v = KeyedVectors.load_word2vec_format('glove.6B.100d.txt.word2vec', binary=False)

In [226]:
import numpy as np
SEQUENCE_LENGTH = 4
HIDDEN_SIZE = 256

EMBEDDING_SIZE_ORIG = 100
EMBEDDING_SIZE = 103

def encode_word(word, w2v):
    if word == "<pad>":
        v = np.zeros((EMBEDDING_SIZE,))
        v[EMBEDDING_SIZE-1] = 1
        return v
    elif word == "<newline>":
        v = np.zeros((EMBEDDING_SIZE,))
        v[EMBEDDING_SIZE-2] = 1
        return v
    elif word == "<unk>" or word not in w2v:
        v = np.zeros((EMBEDDING_SIZE,))
        v[EMBEDDING_SIZE-3] = 1
        return v
    else:        
        v = w2v[word]
        w = np.zeros((3,))
        return np.append(v, w, axis=0)

def encode_words(words, w2v):
    vec = np.zeros((len(words), EMBEDDING_SIZE))
    for (i,word) in enumerate(words):
        vec[i] = encode_word(word, w2v)
    return vec

In [227]:
v = encode_word("hello", w2v)
print(v.shape)
w = encode_word("boom-a-boomerang", w2v)
print(w.shape)

(103,)
(103,)


In [228]:
v_newline = encode_word("<newline>", w2v)
v_pad = encode_word("<pad>", w2v)
v_unk = encode_word("<unk>", w2v)

print(np.argmax(v_newline))
print(np.argmax(v_pad))
print(np.argmax(v_unk))

101
102
100


In [229]:
def decode_vec(vec, w2v):
    base_vec = vec[:EMBEDDING_SIZE_ORIG]
    ext_vec = vec[EMBEDDING_SIZE_ORIG:]
    if ext_vec[0]:
        return "<unk>"
    elif ext_vec[1]:
        return "<newline>"
    elif ext_vec[2]:
        return "<pad>"
    else:
        return w2v.similar_by_vector(base_vec)[0][0]

In [230]:
print(decode_vec(v, w2v))
print(decode_vec(v_newline, w2v))
print(decode_vec(v_pad, w2v))
print(decode_vec(v_unk, w2v))

hello
<newline>
<pad>
<unk>


In [231]:
def prepare_song(song, buffer_length):
    tokens = song

    x_train = []
    y_train = []
    for i in range(0, len(song)):
        if i+buffer_length+1 >= len(tokens):
            pad_length = (i+buffer_length+1) - len(tokens)
            tokens += ['<pad>'] * pad_length

        x_train.append(tokens[i:i+buffer_length])
        y_train.append(tokens[i+buffer_length])

    return x_train,y_train

In [232]:
token_vocab = {'<pad>', '<unk>'}
songs = []
with open("sentences.txt", "r") as f:
    for line in f.readlines():
        tokens = [token for token in line.rstrip().split(" ")]
        songs.append(tokens)
        token_vocab = token_vocab.union(set(tokens))

In [233]:
words = list(token_vocab)
vocab_size = len(words)
print("Vocab size:", vocab_size)
print("W2V vocab size:", len(w2v.vocab))
word2idx = { word:i for i,word in enumerate(words) }
idx2word = { i:word for i,word in enumerate(words) }

Vocab size: 5111
W2V vocab size: 400000


In [234]:
def one_hot_encode(word, word2idx):
    v = np.zeros((len(word2idx, )))
    v[word2idx[word]] = 1
    return v

def one_hot_decode(word, idx2word):
    return idx2word[np.argmax(word)]

In [235]:
print(word2idx["hello"])
print(np.argmax(one_hot_encode("hello", word2idx)))

4886
4886


In [236]:
x_vec = []
y_vec = []
for song in songs:
    x_vec_i, y_vec_i = prepare_song(song, SEQUENCE_LENGTH)
    x_vec.extend(x_vec_i)
    y_vec.extend(y_vec_i)
print(len(x_vec))
print(x_vec[0])

114929
['look', 'at', 'her', 'face']


In [237]:
from random import shuffle
def generate_batches(data_length, mini_batch_size):
    for begin in range(0, data_length, mini_batch_size):
        end = min(begin + mini_batch_size, data_length)
        yield begin, end

def load_batch(xs, ys, begin, end):
    batch_size = end-begin
    
    x_train = np.zeros((batch_size, SEQUENCE_LENGTH, EMBEDDING_SIZE))
    y_train = np.zeros((batch_size, vocab_size))
    
    xs_batch = xs[begin:end]
    ys_batch = ys[begin:end]
    for i in range(batch_size):
        x_train[i] = encode_words(xs_batch[i], w2v)
        y_train[i] = one_hot_encode(ys_batch[i], word2idx)
    
    return x_train, y_train

In [238]:
batches = generate_batches(len(x_vec), 512)
begin, end = next(batches)

x_train, y_train = load_batch(x_vec, y_vec, begin, end)
print(x_train.shape, y_train.shape)

print("X train")
for j in range(10):
    ws = ' '.join([decode_vec(x_train[j][i], w2v) for i in range(SEQUENCE_LENGTH)])
    print(ws)
    
print("\nY train")
for j in range(10):
    print(one_hot_decode(y_train[j], idx2word))

(512, 4, 103) (512, 5111)
X train
look at her face
at her face ,
her face , it
face , it 's
, it 's a
it 's a wonderful
's a wonderful face
a wonderful face <newline>
wonderful face <newline> and
face <newline> and it

Y train
,
it
's
a
wonderful
face
<newline>
and
it
means


In [241]:
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Embedding, Dropout, Bidirectional
def build_model(vocab_size):
    model = Sequential()
    model.add(LSTM(128, input_shape=(SEQUENCE_LENGTH, EMBEDDING_SIZE), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(1024)))
    model.add(Dropout(0.2))
    model.add(Dense(vocab_size, activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer="adam", metrics = ['accuracy'])
    return model
model = build_model(vocab_size)

In [242]:
batches = generate_batches(len(x_vec), 512)
print(len(x_vec) / 512)
for i in range(4):
    begin, end = next(batches)
    x_train, y_train = load_batch(x_vec, y_vec, begin, end)
    model.fit(x_train, y_train, batch_size=128, epochs=10)

224.470703125
Epoch 1/10
512/512 [==============================] - 2s 4ms/step - loss: 8.4023 - accuracy: 0.0801
Epoch 2/10
512/512 [==============================] - 1s 2ms/step - loss: 6.5219 - accuracy: 0.1113
Epoch 3/10
512/512 [==============================] - 1s 2ms/step - loss: 4.7103 - accuracy: 0.0664
Epoch 4/10
512/512 [==============================] - 1s 2ms/step - loss: 4.5463 - accuracy: 0.0254
Epoch 5/10
512/512 [==============================] - 1s 2ms/step - loss: 4.3643 - accuracy: 0.0781
Epoch 6/10
512/512 [==============================] - 1s 2ms/step - loss: 4.2246 - accuracy: 0.1113
Epoch 7/10
512/512 [==============================] - 1s 2ms/step - loss: 4.1796 - accuracy: 0.1113
Epoch 8/10
512/512 [==============================] - 1s 2ms/step - loss: 4.1252 - accuracy: 0.1113
Epoch 9/10
512/512 [==============================] - 1s 2ms/step - loss: 4.0863 - accuracy: 0.1230
Epoch 10/10
512/512 [==============================] - 1s 2ms/step - loss: 4.0272 - ac

In [244]:
words = ["look", "at", "her", "face"]
#words = ["do", "better", ",", "who", "better", "?", "<newline>", "you"]
words_seq = encode_words(words, w2v)
words_seq = words_seq.reshape(1, SEQUENCE_LENGTH, EMBEDDING_SIZE)
#print(' '.join([decode_vec(words[0][i], w2v) for i in range(SEQUENCE_LENGTH)]))

result = words
for j in range(60):
    word = one_hot_decode(model.predict(words_seq), idx2word)
    result.append(word)
    
    new_words = np.zeros((1, SEQUENCE_LENGTH, EMBEDDING_SIZE))
    for i in range(SEQUENCE_LENGTH-1):
        new_words[0, i] = words_seq[0, i+1]
    new_words[0, SEQUENCE_LENGTH-1] = encode_word(word, w2v)
    words_seq = new_words

#print(' '.join([decode_vec(words[0][i], w2v) for i in range(SEQUENCE_LENGTH)]))
 
print(' '.join(result))
    #words = new_words
        #new_words[0] = words[0, 1]
        #new_words[0, 1] = words[0, 2]
        #new_words[0, 2] = words[0, 3]
        #new_words[0, 3] = encode_word(word, word2idx)
    

look at her face <newline> <newline> <newline> <newline> oh peace peace now <pad> would would you to you to <newline> words <newline> we the <newline> the <newline> <newline> <newline> <newline> oh peace peace now <pad> would would you to you to <newline> words <newline> we the <newline> the <newline> <newline> <newline> <newline> oh peace peace now <pad> would would you to you to <newline>
